<a href="https://colab.research.google.com/github/bansalhim/Deep-learning-assignment/blob/main/Assignment3_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Logistic Regression as a Soft Decision**

In [ ]:
import pandas as pd
#load daataset
df=pd.read_csv('/content/glass.csv')
# Inspect the dataset
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
df.head()# now getting values in daataset

Shape: (214, 10)
Columns: ['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type']


,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [ ]:
# Step 2: Inspecting Data

# The output column we want to predict is 'Type'.
# All other columns are numeric values of chemical components.

print("Data types:\n", df.dtypes)# printiig datatype which is floating values in dataset
print("\nSummary statistics:\n", df.describe())


Data types:
 RI      float64
Na      float64
Mg      float64
Al      float64
Si      float64
K       float64
Ca      float64
Ba      float64
Fe      float64
Type      int64
dtype: object

Summary statistics:
                RI          Na          Mg          Al          Si           K  \
count  214.000000  214.000000  214.000000  214.000000  214.000000  214.000000   
mean     1.518365   13.407850    2.684533    1.444907   72.650935    0.497056   
std      0.003037    0.816604    1.442408    0.499270    0.774546    0.652192   
min      1.511150   10.730000    0.000000    0.290000   69.810000    0.000000   
25%      1.516522   12.907500    2.115000    1.190000   72.280000    0.122500   
50%      1.517680   13.300000    3.480000    1.360000   72.790000    0.555000   
75%      1.519157   13.825000    3.600000    1.630000   73.087500    0.610000   
max      1.533930   17.380000    4.490000    3.500000   75.410000    6.210000   

               Ca          Ba          Fe        Type  
count

In [ ]:
# Step 3: Binary classification setup
# Convert 'Type' into a binary target variable: as type is output so type is given 1 and rest are given 0 value
# Type 1 -> positive class (1)
# All other types -> negative class (0)

df["y"] = (df["Type"] == 1).astype(int)

# Drop the original Type column
df = df.drop(columns=["Type"])

# Verify
df.head()
#Why this is done
#because Logistic regression handles binary output only.

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,y
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [ ]:
# Step 4: Split into features and label
# means from this i get is now we are spliting of seprating input which is x and output which is y
X = df.drop(columns=["y"]).values
y = df["y"].values

print("X shape:", X.shape)
print("y shape:", y.shape)
#i observe that the model learns a mapping from inputs X to output y.

X shape: (214, 9)
y shape: (214,)


In [ ]:
# Step 5: Split into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])
# so total we have 214 so it split into training nad testing 171  for training and 43 for testing
# as test size is 0.2

Training samples: 171
Testing samples: 43


In [ ]:
# Step 6: Feature Scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# Scaling ensures all features have zero mean and unit variance.
# Without scaling, features with larger numeric ranges dominate training.
# so we do not want that for that scaling is performed
# so mean is 0 and sd is 1 .Scaling ensures all features are equal in importance.

In [ ]:
# Step 7: Define Sigmoid
import numpy as np

def sigmoid(z):
    """Compute sigmoid activation."""
    return 1 / (1 + np.exp(-z))
## Now i use sigmoid function becauseit gives a smooth probability between 0 and 1.
# and for complex data having noice sigmoid is used

In [ ]:
# Step 8: Compute predicted probabilities-
def predict_proba(X, w, b):
    z = X @ w + b#equation is used in this
    p = sigmoid(z)
    return p


In [ ]:
# Step 9: Define loss function

def loss(y, p):
    """Binary cross-entropy loss."""
    return -np.mean(y * np.log(p + 1e-8) + (1 - y) * np.log(1 - p + 1e-8))
#so large penalty for confident wrong predictions. as logistic tell how correct confidence is and if confidence is wrong so large penatly is made and Cross-entropy measures how wrong the predicted probabilities are.

In [ ]:
# Step 10: Weight update rule->Gradient Descent
def update_weights(X, y, w, b, lr):
    p = predict_proba(X, w, b)
    error = p - y
    w = w - lr * (X.T @ error) / len(y)
    b = b - lr * np.mean(error)
    return w, b
##  error (p - y) indicates both direction and magnitude of correction
# llearning rate controls how fast we adjust weights.
#This is the learning step — it reduces the loss step-by-step.

In [ ]:
# Step 11: Training loop
w = np.zeros(X_train.shape[1])
b = 0.0
lr = 0.1
epochs = 100

for epoch in range(epochs):
    p_train = predict_proba(X_train, w, b)
    w, b = update_weights(X_train, y_train, w, b, lr)
    if epoch % 10 == 0:
        current_loss = loss(y_train, p_train)
        print(f"Epoch {epoch:3d}, Loss: {current_loss:.6f}")

print("Training complete!")
# loss should decrease with epochs
# If it increases it meansthat learning rate too high or gradient incorrect.
#This runs training many times (100 epochs).
#Each round will predict then calculate loss and then update weights.

Epoch   0, Loss: 0.693147
Epoch  10, Loss: 0.615693
Epoch  20, Loss: 0.577566
Epoch  30, Loss: 0.554707
Epoch  40, Loss: 0.539208
Epoch  50, Loss: 0.527877
Epoch  60, Loss: 0.519169
Epoch  70, Loss: 0.512233
Epoch  80, Loss: 0.506557
Epoch  90, Loss: 0.501812
Training complete!


In [ ]:
# Step 12: Convert probabilities to class labels
def predict_label(p, threshold=0.5):
    return (p >= threshold).astype(int)

# Evaluate on test data
p_test = predict_proba(X_test, w, b)
y_pred_05 = predict_label(p_test, 0.5)
y_pred_07 = predict_label(p_test, 0.7)

from sklearn.metrics import accuracy_score, confusion_matrix

print("Accuracy (threshold 0.5):", accuracy_score(y_test, y_pred_05))
print("Accuracy (threshold 0.7):", accuracy_score(y_test, y_pred_07))
print("Confusion matrix (0.5):\n", confusion_matrix(y_test, y_pred_05))
# Higher threshold (0.7) reduces false positives but may increase false negatives.
# In quality control, it's safer to label uncertain samples as "not Type 1."
#If the model is >50% confident → classify as Type 1.
#we can change threshold to make it stricter or looser.

Accuracy (threshold 0.5): 0.8604651162790697
Accuracy (threshold 0.7): 0.7209302325581395
Confusion matrix (0.5):
 [[30  2]
 [ 4  7]]


In [ ]:

# The model achieved higher accuracy at threshold 0.5 (86%) than at 0.7 (72%).
# icreasing the threshold makes the model more conservative in predicting "Type 1."
# As a result, it predicts fewr positives (reduces false positives) but misses some actual positives (increases false negatives).
# This is expected behavior — a higher threshold increases precision but decreases recall.
# In a real-world qulity control setting, such as identifying a rare glass type,
# we may prefer a higher threshold to redue false alarms, even at the cost of missing some positive.


In [ ]:
from sklearn.metrics import precision_score, recall_score

print("Precision (0.5):", precision_score(y_test, y_pred_05))
print("Recall (0.5):", recall_score(y_test, y_pred_05))
print("Precision (0.7):", precision_score(y_test, y_pred_07))
print("Recall (0.7):", recall_score(y_test, y_pred_07))


Precision (0.5): 0.7777777777777778
Recall (0.5): 0.6363636363636364
Precision (0.7): 0.0
Recall (0.7): 0.0


In [ ]:

# At threshold = 0.5, the model predicts some positive (Type 1) samples correctly,
# giving decent precision (0.78) and recall (0.64).
# When threshold increases to 0.7, the model becomes too strict —
# no sample reaches 0.7 probability, so it predicts all as "Not Type 1".
# Therefore, both precision and recall drop to 0.
# This shows how threshold tuning controls the trade-off:
# lower threshold → more positives (higher recall),
# higher threshold → fewer positives (higher precision, but only if any positives remain).
# In our case, threshold 0.7 is too high for this dataset.


In [ ]:
# Step 13: Example probability inspection
sample_probs = p_test[:10]
print("Sample predicted probabilities:\n", sample_probs)


Sample predicted probabilities:
 [0.46596617 0.02699184 0.60769832 0.02225283 0.30880036 0.30065433
 0.5507444  0.44201251 0.43843704 0.40641316]


Step 14 –

**1. How this differs from a perceptron?**

The perceptron outputs a hard 0/1 decision using a step function.
Logistic regression give outputs a continuous probability using the sigmoid function.
This allows it to represent uncertainty and to train using gradient-based optimization.and also if we have noisy data logistic handles more better then perceptron.

**2. Why the sigmoid matters?**

The sigmoid converts the linear combination of inputs (evidence z) into a bounded value between 0 and 1.
This makes the output interpretable as probability and keeps the gradient smooth and differentiable for learning instead of getting hard binary values.

3. What problem still remains unsolved?

Even though the model learns probabilities, it still assumes a linear boundary between classes.
It cannot capture complex or nonlinear relationships — something that later neural networks with multiple layers and nonlinear activations will address.